# Decision Tree

## Introdução às Decision Trees

As **Decision Trees** são algoritmos de aprendizagem supervisionada utilizados em tarefas de **classificação** e **regressão**. Funcionam através de uma divisão recursiva dos dados com base em regras de decisão simples, formando uma estrutura em forma de árvore.

Cada divisão é guiada por uma métrica de qualidade, sendo a mais comum o **Information Gain**, que mede a redução de **entropia** após uma divisão, ou seja, o quanto uma determinada feature contribui para separar eficazmente as classes.

## Aplicação no Projeto

Neste projeto, utilizamos uma árvore de decisão para **prever a jogada ótima** (i.e., a coluna a jogar) a partir de um estado do tabuleiro no jogo *4 em linha*. A jogada considerada ótima é aquela que seria escolhida por um agente **Monte Carlo Tree Search (MCTS)** com **10.000 iterações**.

## Geração do Dataset

Para treinar a árvore de decisão, foi gerado um **dataset supervisionado** com aproximadamente **500 jogos simulados**, a partir dos quais foram extraídas diversas posições intermédias com respetiva jogada ideal anotada.

## Evolução da Representação do Dataset

### Estrutura Inicial

Na fase inicial do projeto, o dataset foi construído com base numa **representação direta do estado do tabuleiro** do jogo *4 em linha*. Cada linha do dataset correspondia a um momento específico de um jogo simulado, e era composta pelos seguintes elementos:

- **Um único vetor** que representava o tabuleiro completo:
  - Cada célula da grelha era codificada como:
    - `0` → célula vazia
    - `1` → peça do jogador 1
    - `-1` → peça do jogador 2
  - A ordem dos elementos era **coluna a coluna**, **do topo para a base**, espelhando a forma como as peças são empilhadas no jogo.
- Uma coluna adicional com o **número total de peças** jogadas até ao momento (proxy da profundidade no jogo).
- Uma coluna que indicava o **jogador atual** a jogar (1 ou 2).
- A coluna de **output** representava a **coluna ideal** para jogar, determinada por um agente **Monte Carlo Tree Search (MCTS)** com 10.000 iterações.

Esta estrutura simples era funcional e suficiente para uma primeira abordagem, mas apresentava algumas **limitações**, nomeadamente:

- Mistura da informação dos dois jogadores num único vetor, dificultando a distinção clara entre as peças.
- Potencial ambiguidade para modelos de aprendizagem que tratam os dados de forma vetorial.
- Menor capacidade de generalização para diferentes representações.

---



### Nova Estrutura do Dataset

Inspirado por abordagens usadas no treino de agentes para o jogo **Go**, a representação do estado do jogo foi reformulada para melhor refletir a informação posicional de forma explícita e neutra. O estado do tabuleiro é agora representado da seguinte forma:

- **Dois tabuleiros binários**, cada um com as mesmas dimensões do tabuleiro original (por exemplo, 6x7):
  - Um tabuleiro representa as posições ocupadas pelo **Jogador 1** (`1` para peça presente, `0` caso contrário).
  - O outro representa as posições do **Jogador 2**, com a mesma codificação.
- Cada entrada do dataset inclui:
  - Os dois vetores resultantes do **flattening** dos tabuleiros.
  - Uma feature adicional com o **número total de peças** jogadas até ao momento.
  - Uma feature que identifica o **jogador atual** (1 ou -1).
  - A **coluna de output** com a jogada ideal sugerida pelo MCTS (com 10.000 simulações).

Esta nova representação não só oferece uma **separação clara da informação entre os dois agentes**, como também permite que o modelo **aprenda padrões estratégicos específicos de cada jogador**. A estrutura binária facilita ainda a integração com modelos baseados em regras e em redes neuronais, caso seja pretendida uma extensão futura do projeto.

---

Na próxima secção será discutida a implementação técnica do algoritmo ID3 utilizado para treinar a árvore de decisão com base neste dataset.


In [ ]:
import pandas as pd
df = pd.read_csv('datasets/monte_carlo_data.csv',delimiter=';')
display(df.head())

,cel1,cel2,cel3,cel4,cel5,cel6,cel7,cel8,cel9,cel10,...,cel36,cel37,cel38,cel39,cel40,cel41,cel42,pieces,turn,played
0,0,0,0,0,0,0,0,0,0,0,...,0,-1,-1,1,1,-1,1,12,1,5.0
1,0,0,0,0,0,0,0,0,0,0,...,0,-1,1,1,-1,-1,-1,13,1,5.0
2,0,0,0,0,0,0,0,0,0,0,...,0,-1,-1,1,-1,1,1,7,1,5.0
3,0,0,0,0,0,-1,0,0,0,0,...,-1,1,0,1,-1,1,-1,17,1,3.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,-1,1,5,1,6.0



 

# Tentativa de Randomização das Posições

Durante a fase de geração do dataset, foi inicialmente considerada a ideia de **randomizar posições do tabuleiro** em vez de as obter exclusivamente a partir de jogos completos simulados com agentes MCTS.

A motivação por trás desta abordagem era: 
- Aumentar a diversidade de posições no dataset
- Reduzir o tempo necessário para simular jogos completos
- Explorar casos de jogo menos prováveis, mas ainda legais

### Estratégia Testada

O processo consistia em:
1. Gerar posições aleatórias válidas (respeitando as regras do 4 em linha, como gravidade das peças)
2. Avaliar essas posições usando o agente MCTS com 10.000 iterações
3. Adicionar a posição e a jogada recomendada ao dataset

Apesar de parecer viável, esta abordagem foi **desaconselhada pelo professor da cadeira** a qual foi, então, descartada.



# Algoritmo ID3 - Implementação da Árvore de Decisão

Neste capítulo será abordada a implementação do algoritmo **ID3** para construção de Decision Trees , conforme desenvolvido no ficheiro [ID3Tree.py](DecisionTree/ID3Tree.py).

A implementação encontra-se encapsulada numa classe denominada `ID3Tree`, a qual integra os métodos essenciais para:
- cálculo da entropia de um conjunto de rótulos,
- determinação do ganho de informação,
- construção recursiva da árvore de decisão,
- classificação de novos exemplos.



## Método `entropy`

O método `entropy` é definido na classe `ID3Tree` e tem como função calcular a **entropia** de um conjunto de rótulos, o que corresponde a uma medida quantitativa da **impureza** ou incerteza inerente a esse conjunto.

```python
def entropy(self, labels):
    """
    Calculate the entropy of a set of labels.
    - labels: List of class labels.
    """
    total = len(labels)
    counter = Counter(labels)  # Count occurrences of each label
    return -sum((count / total) * math.log2(count / total) for count in counter.values())
```

### Definição e Justificação

A entropia é uma métrica fundamental na teoria da informação, utilizada para quantificar a quantidade de incerteza num conjunto de dados. No contexto do algoritmo ID3, é empregue para medir a heterogeneidade dos rótulos em cada subconjunto de dados, servindo de base para a escolha dos atributos que melhor segmentam a informação.

Formalmente, a entropia $H(S)$ de um conjunto $S$ contendo $C$ classes distintas é dada por:

$$
H(S) = - \sum_{i=1}^{C} p_i \log_2(p_i)
$$

onde $p_i$ representa a proporção de elementos pertencentes à classe $i$ no conjunto $S$.

### Análise do Método

- **Cálculo do total de amostras**:  
  O número total de rótulos no conjunto é determinado através de `total = len(labels)`.

- **Contagem da frequência de cada classe**:  
  Utiliza-se a estrutura `Counter` da biblioteca `collections` para obter a frequência absoluta de cada classe.

- **Cálculo da entropia**:  

  Para cada classe, calcula-se a frequência relativa  $ p_i = \frac{\text{count}}{\text{total}} $ e avalia-se o termo $ -p_i \log_2(p_i) $. A soma destes termos para todas as classes resulta no valor da entropia do conjunto.

### Relevância para o Algoritmo ID3

O cálculo da entropia é indispensável para o cálculo subsequente do **ganho de informação** (information gain), que avalia a eficácia da segmentação do conjunto de dados por cada atributo. O atributo que proporciona a maior redução da entropia é escolhido para a divisão do nó, conduzindo a uma árvore de decisão mais eficaz e informativa.



## Método `id3_train`

O método `id3_train` é o núcleo da construção da árvore de decisão, implementando o algoritmo **ID3** de forma recursiva. Este método recebe como entrada o conjunto de dados de treino e a lista de atributos disponíveis para a divisão, e devolve a estrutura da árvore construída.

```python
def id3_train(self, data, attributes):
    """
    Recursively build the decision tree using the ID3 algorithm.
    - data: Training data.
    - attributes: List of attributes to consider.
    """
    if not data:
        return self.default  # Return default if no data is available
    if len(set(row[-1] for row in data)) == 1:
        return data[0][-1]  # Return the label if all data has the same label

    # Calculate fitness scores for all attributes
    scores = [(self.fitness_for(attr)(data, attr), attr) for attr in attributes]
    best_gain, best_attr = max(scores, key=lambda x: x[0][0] if isinstance(x[0], tuple) else x[0])

    if self.type_map[best_attr] == 'continuous':
        # Handle continuous attributes
        threshold = best_gain[1]
        node = Node(best_attr, threshold, best_gain[0])  # Create a node with a threshold
        above = [row for row in data if row[self.attributes.index(best_attr)] >= threshold]
        below = [row for row in data if row[self.attributes.index(best_attr)] < threshold]
        return {node: {
            '>=': self.id3_train(above, attributes),
            '<': self.id3_train(below, attributes)
        }}
    else:
        # Handle discrete attributes
        index = self.attributes.index(best_attr)
        values = set(row[index] for row in data)
        node = Node(best_attr, None, best_gain[0])  # Create a node without a threshold
        return {node: {
            val: self.id3_train([row for row in data if row[index] == val], [a for a in attributes if a != best_attr])
            for val in values
        }}
```



### Explicação detalhada

1. **Condição de paragem: ausência de dados**

   ```python
   if not data:
       return self.default
   ```

   Caso o conjunto de dados recebido esteja vazio, a função retorna a classe padrão `self.default`, que normalmente corresponde à classe mais frequente do conjunto de treino inicial. Esta condição impede que o algoritmo tente construir uma árvore com dados inexistentes.

2. **Condição de paragem: pureza do nó**

   ```python
   if len(set(row[-1] for row in data)) == 1:
       return data[0][-1]
   ```

   Se todos os exemplos no conjunto de dados têm o mesmo rótulo (ou seja, o nó é puro), o método retorna esse rótulo. Neste ponto, a construção da árvore para naquele ramo, pois a decisão já está clara.

3. **Cálculo do ganho de informação para cada atributo**

   ```python
   scores = [(self.fitness_for(attr)(data, attr), attr) for attr in attributes]
   best_gain, best_attr = max(scores, key=lambda x: x[0][0] if isinstance(x[0], tuple) else x[0])
   ```

   Para cada atributo na lista de atributos disponíveis, calcula-se a sua "fitness" que corresponde ao ganho de informação (information gain) associado à divisão pelo atributo.  
   
   O método `fitness_for(attr)` retorna a função apropriada para calcular a qualidade da divisão segundo o tipo do atributo (contínuo ou discreto).  
   
   De seguida, seleciona-se o atributo que maximiza o ganho de informação, pois este atributo permitirá a melhor segmentação dos dados naquele nó.

4. **Divisão do conjunto de dados consoante o tipo do atributo**

   - **Atributos contínuos**

     Quando o atributo é contínuo, é determinado um limiar (`threshold`) que melhor divide o conjunto.

     ```python
     threshold = best_gain[1]
     node = Node(best_attr, threshold, best_gain[0])
     ```

     Criamos um nó contendo o atributo, o limiar e o valor do ganho.  
     
     Depois, os dados são particionados em dois subconjuntos:
     - `above`: exemplos cujo valor do atributo é maior ou igual ao limiar,
     - `below`: exemplos cujo valor do atributo é inferior ao limiar.

     Para cada subconjunto, chama-se recursivamente `id3_train` para continuar a construção da árvore:

     ```python
     return {node: {
         '>=': self.id3_train(above, attributes),
         '<': self.id3_train(below, attributes)
     }}
     ```

   - **Atributos discretos**

     Quando o atributo é discreto, o conjunto de dados é dividido em tantos subconjuntos quanto os valores distintos do atributo:

     ```python
     index = self.attributes.index(best_attr)
     values = set(row[index] for row in data)
     node = Node(best_attr, None, best_gain[0])
     ```

     Para cada valor distinto do atributo, filtra-se o conjunto de dados correspondente e chama-se recursivamente `id3_train` excluindo o atributo já utilizado para evitar ciclos.

     ```python
     return {node: {
         val: self.id3_train([row for row in data if row[index] == val], [a for a in attributes if a != best_attr])
         for val in values
     }}
     ```



## Tratamento de dados contínuos vs discretos

Embora o dataset principal do projeto, correspondente aos jogos de 4 em linha, seja composto exclusivamente por atributos discretos à exeção do atributo do nr de peças, foi solicitado no âmbito do trabalho, que a árvore de decisão fosse testada também no conjunto de dados clássico **Iris**, que possui atributos contínuos.

Por esta razão, tornou-se imperativo que a função `id3_train` distinguisse explicitamente entre atributos contínuos e discretos, aplicando abordagens específicas para cada caso:

- Para atributos contínuos, é necessário determinar o melhor limiar (`threshold`) para a divisão do conjunto, partindo os dados em subconjuntos baseados nesse ponto de corte.
- Para atributos discretos, o conjunto é particionado em subconjuntos de acordo com os valores categóricos presentes.


### Função `id3_continuous`

Esta função avalia o ganho de informação associado a um **atributo contínuo**.

```python
def id3_continuous(self, data, attribute):
    """
    Calculate the information gain for a continuous attribute.
    - data: Training data.
    - attribute: The attribute to evaluate.
    """
    idx = self.attributes.index(attribute)
    values = sorted(set(row[idx] for row in data))  # Unique sorted values of the attribute
    if len(values) == 1:
        return -1, None  # No split possible if only one unique value

    # Calculate potential thresholds
    thresholds = [(values[i] + values[i + 1]) / 2 for i in range(len(values) - 1)]
    base_entropy = self.entropy([row[-1] for row in data])  # Entropy of the entire dataset

    best_gain, best_thresh = -1, None
    for t in thresholds:
        # Split data into above and below threshold
        above = [row for row in data if row[idx] >= t]
        below = [row for row in data if row[idx] < t]
        p, n = len(above) / len(data), len(below) / len(data)
        # Calculate information gain
        gain = base_entropy - p * self.entropy([r[-1] for r in above]) - n * self.entropy([r[-1] for r in below])
        if gain > best_gain:
            best_gain, best_thresh = gain, t
    return best_gain, best_thresh
```

#### Explicação técnica

1. **Extração e ordenação dos valores do atributo**

   ```python
   values = sorted(set(row[idx] for row in data))
   ```

   São considerados apenas os valores únicos e ordenados do atributo contínuo.

2. **Geração de limiares candidatos**

   São gerados todos os possíveis pontos médios entre pares consecutivos de valores:

   $$
   \text{threshold}_i = \frac{v_i + v_{i+1}}{2}
   $$

   Estes limiares são os pontos de corte candidatos para dividir o conjunto de dados.

3. **Cálculo do ganho de informação**

   Para cada limiar $ t $, divide-se o conjunto em duas partes:
   - Acima do limiar: $ D_{\geq t} $
   - Abaixo do limiar: $ D_{< t} $

   O ganho de informação é então calculado como:

   $$
   \text{Gain}(t) = H(D) - p \cdot H(D_{\geq t}) - (1 - p) \cdot H(D_{< t})
   $$

   onde $ H(D) $ é a entropia do conjunto total e $ p $ é a proporção de dados em $ D_{\geq t} $.

4. **Resultado**

   A função devolve o limiar $ t $ que gera o maior ganho de informação, juntamente com o valor desse ganho.



### Função `id3_discrete`

Esta função calcula o ganho de informação para atributos **discretos**, ou seja, com um número finito de categorias.

```python
def id3_discrete(self, data, attribute):
    """
    Calculate the information gain for a discrete attribute.
    - data: Training data.
    - attribute: The attribute to evaluate.
    """
    idx = self.attributes.index(attribute)
    base_entropy = self.entropy([row[-1] for row in data])  # Entropy of the entire dataset
    values = set(row[idx] for row in data)  # Unique values of the attribute

    remainder = 0
    for val in values:
        # Subset of data where the attribute equals the current value
        subset = [row for row in data if row[idx] == val]
        remainder += (len(subset) / len(data)) * self.entropy([row[-1] for row in subset])

    return base_entropy - remainder, None
```

#### Explicação técnica

Neste caso, o conjunto de dados é particionado em subconjuntos distintos consoante os valores únicos do atributo.

O ganho de informação é calculado da seguinte forma:

- Entropia inicial: $ H(D) $
- Resto (_remainder_):

  $$
  \text{Remainder}(A) = \sum_{v \in \text{Values}(A)} \frac{|D_v|}{|D|} \cdot H(D_v)
  $$

  onde $ D_v $ representa o subconjunto de dados para os quais o atributo $ A = v $.

- Finalmente, o ganho é:

  $$
  \text{Gain}(A) = H(D) - \text{Remainder}(A)
  $$

Como não há limiar em atributos discretos, o segundo valor devolvido pela função é `None`.


## Geração de Regras a Partir da Árvore - `build_rules`

Uma das grandes vantagens da utilização de Decision Trees  é a sua **capacidade de explicação**. A função `build_rules` tem como objetivo transformar a árvore gerada pelo algoritmo ID3 numa **lista de regras legíveis**, onde cada regra corresponde a um caminho da raiz até uma folha, com as respetivas condições e classificação final.

```python
def build_rules(self, tree=None, premises=None):
    """
    Build a list of rules from the decision tree.
    - tree: The decision tree (default is the trained tree).
    - premises: List of premises leading to the current node.
    """
    tree = self.tree if tree is None else tree
    premises = premises or []
    rules = []

    for node, branches in tree.items():
        for value, subtree in branches.items():
            # Add the current condition to the premises
            new_premise = premises + [(node.attribute, value, node.threshold) if node.threshold is not None else (node.attribute, '=', value)]
            if isinstance(subtree, dict):
                # Recursively build rules for subtrees
                rules.extend(self.build_rules(subtree, new_premise))
            else:
                # Create a rule for a leaf node
                rules.append(Rule(self.attributes, new_premise, subtree))
    return rules
```



### Explicação Técnica

#### Parâmetros:

- **`tree`**: Subárvore atual. Caso não seja fornecida, utiliza-se a árvore completa treinada (`self.tree`).
- **`premises`**: Lista de condições acumuladas ao longo do caminho da raiz até ao nó atual. Cada condição é armazenada como uma tupla.

#### Objetivo:

Gerar uma lista de objetos da classe `Rule`, onde cada objeto representa uma regra da forma:

> **SE** (atributo1 = valor1) **E** (atributo2 ≥ threshold2) **ENTÃO** classe = X



### Processo Recursivo

1. **Iterar sobre os nós da árvore**:
   A árvore é representada como um dicionário onde cada chave é um `Node` e os valores são os ramos descendentes desse nó.

2. **Construção de condições (premises)**:
   Cada nó adiciona uma nova condição à lista de `premises`. A condição é construída de forma diferente consoante se trata de um atributo contínuo (com `threshold`) ou discreto.

   - Para contínuos:  
     ```python
     (atributo, operador, threshold)
     ```
     onde `operador` será `'<'` ou `'>='` consoante o ramo.

   - Para discretos:
     ```python
     (atributo, '=', valor)
     ```

3. **Verificação do tipo de ramo**:
   - Se o ramo ainda for um dicionário (`dict`), significa que há mais subdivisões, e a função é chamada recursivamente.
   - Se for um valor (rótulo), significa que foi alcançada uma **folha**, e uma nova regra é criada com o conjunto atual de premissas.



### Exemplo de Regra Gerada

Suponhamos que a árvore contenha os seguintes ramos:

- `Node(attribute='coluna_1', threshold=0.5)`:
  - Ramo `>=`: vai para `Node(attribute='coluna_3', threshold=None)`
    - Ramo `'=': 1` → classe `Jogador_1`
    - Ramo `'=': 0` → classe `Jogador_2`

Neste caso, as regras geradas seriam algo do género:

- SE `coluna_1 ≥ 0.5` E `coluna_3 = 1` → `Classe = Jogador_1`
- SE `coluna_1 ≥ 0.5` E `coluna_3 = 0` → `Classe = Jogador_2`


## Conclusão - Algoritmo ID3

A aplicação do algoritmo **ID3** no contexto do jogo *4 em linha* permitiu uma primeira aproximação à criação de um modelo supervisionado com base em regras explícitas. Apesar da sua simplicidade, o ID3 revelou-se pouco eficaz quando aplicado diretamente sobre o **dataset derivado de estados de jogo simulados por MCTS**.



### Resultados e Observações

Durante os testes realizados, o modelo alcançou uma **accuracy entre 30% a 40% em dados de teste**, enquanto apresentava **valores superiores a 90% nos dados de treino**. Este comportamento revela a ocorrência de dois problemas fundamentais:

- **Overfitting (Alta Variância)**: O modelo ajusta-se excessivamente aos dados de treino, perdendo a capacidade de generalizar para novas situações.
- **Alta Bias estrutural**: A simplicidade do ID3 impossibilita uma estratégia mais complexa necessária para uma boa árvore de decisão no jogo de 4 em linha, especialmente em cenários com múltiplas interdependências entre jogadas.



### Considerações sobre a Natureza do Problema

Estes resultados eram **esperados**, dado que:

- O jogo *4 em linha* é altamente estratégico e **não-linear**, com muitas combinações possíveis de jogadas dependentes do contexto.
- O ID3 **não tem memória nem profundidade estratégica**, baseando-se apenas em partições de dados locais, sem considerar consequências futuras.



### Caminhos Futuros

De forma a **ultrapassar as limitações** observadas com o ID3, foram exploradas outras abordagens mais robustas e adequadas à natureza do problema:

- **Bagging (Bootstrap Aggregation)**
- **RuleSet Generalization** 



#  RuleSet com Pruning

## Introdução Teórica

Um **RuleSet** é uma representação de um modelo de decisão na forma de um conjunto explícito de **regras if-then**, derivadas geralmente de modelos base como **Decision Trees **.

No contexto de Supervised Learning, as Decision Trees  como o ID3 podem ser **transformadas num conjunto de regras**, onde cada caminho da raiz até uma folha representa uma regra lógica que conduz a uma classe.



### Problemas dos RuleSets Diretos

Apesar de serem interpretáveis, os RuleSets extraídos diretamente de árvores profundas podem apresentar **grande complexidade e redundância**, o que resulta em:

- **Overfitting** - regras demasiado específicas ao conjunto de treino;
- **Baixa generalização** - fraca performance em dados nunca antes vistos;
- **Dificuldade de manutenção** e análise do modelo.



## Pruning (Poda)

A técnica de **poda** consiste em **remover condições irrelevantes ou pouco impactantes** das regras extraídas, com o objetivo de **simplificar o RuleSet** sem comprometer (e muitas vezes melhorando) a sua performance.

A poda atua como um mecanismo de **regularização**, reduzindo a complexidade do modelo e prevenindo o overfitting. Esta simplificação é feita através de critérios como:

- Acurácia em dados de validação;
- Frequência da regra no dataset;
- Impacto marginal da condição na previsão.



## Aplicação no Projeto

Neste projeto, os RuleSets são derivados das árvores ID3 construídas anteriormente. Para contornar o problema de **complexidade excessiva das regras** e **baixa generalização**, é aplicado um processo de **poda heurística**, que filtra as condições menos relevantes com base em testes de desempenho sobre dados de treino e validação.

Esta abordagem procura um equilíbrio entre **simplicidade** e **eficácia**, e será detalhada nos tópicos seguintes com o respetivo código de implementação e análise dos resultados obtidos.


# Bagging (Bootstrap Aggregation)

## Introdução Teórica

O **Bagging** (*Bootstrap Aggregation*) é uma técnica de aprendizagem em conjunto (**ensemble learning**) cujo principal objetivo é **reduzir a variância** de modelos de machine learning instáveis, como as Decision Trees .

A ideia central do Bagging consiste em:

1. **Gerar múltiplos subconjuntos de dados** a partir do conjunto de treino original, usando amostragem com reposição (bootstrap).
2. **Treinar modelos independentes** sobre cada um desses subconjuntos.
3. **Combinar os resultados** dos modelos.

Esta abordagem ajuda a mitigar o problema de **overfitting** típico em modelos altamente sensíveis aos dados, como é o caso do ID3, ao agregar vários modelos que, embora individualmente imperfeitos, se complementam mutuamente.



### Justificação da Aplicação no Projeto

Como demonstrado no capítulo anterior, a aplicação direta do algoritmo **ID3** ao problema do jogo *4 em linha* resultou numa performance limitada, com clara evidência de **alta variância** - acurácia de treino elevada, mas fraca generalização nos dados de teste.

Assim, a técnica de Bagging surge como uma **tentativa natural de aumentar a robustez do modelo** sem alterar o classificador base. Através da combinação de várias árvores ID3 treinadas em subconjuntos diferentes do dataset, esperamos atingir uma **melhoria significativa da performance**, reduzindo a variância sem comprometer em demasia o viés.

Nos próximos tópicos será apresentada a implementação desta técnica, bem como os resultados obtidos e comparações com o modelo ID3 isolado.
